In [1]:
import os
import glob
import re
import pickle

import numpy as np
import tifffile
import scipy.ndimage as ndi
from matplotlib import pyplot as plt
from skimage import measure
from skimage.segmentation import clear_border
import pandas as pd
import seaborn as sns
from nd2reader import ND2Reader
import plotly.express as px

Pixel size in x and y is .129 micrometers for 60x

In [2]:
# has the wrong name
pickles = sorted(glob.glob("*.pkl"))

dflist = list()
for pk in pickles:
    _df = pd.read_pickle(pk)
    _df['pk_file'] = os.path.basename(pk) 
    dflist.append(_df)
alldf = pd.concat(dflist).reset_index()

In [3]:
len(alldf)

580

In [4]:
df = alldf.copy()

In [ ]:
'''
Mutant names are based on the name of folders in the path and the filename.
The same for replicate names
'''

def mutant_map(x):
    bn = os.path.basename(x)
    for m in mutants:
        if m in bn:
            mutant = m
            break
        
    return m

def rep_map(x):
    bn = os.path.basename(x)
    if 'glucose-2' in bn:
        rep = 'Rep2'
    else:
        rep = 'Rep1'
    
    return rep

def file_map(x):
    s = x.split("/")
    return s[-1]

def path_map(x):
    s = os.path.split(x)[0]
    return s.split("/")[-1]

df['mutant'] = df['file'].map(mutant_map)
df['m2'] = df['mutant'].copy()
df['rep'] = df['file'].map(rep_map)
df['path'] = df['file'].map(path_map)
#df['file'] = df['file'].map(file_map)
df['filename'] = df['file'].str.split("/").str[-1]
df.sample(10)[['file', 'mutant', 'rep', 'path', 'filename']]

In [9]:
mask_dir = "Data/Masks/"
s = mask_dir + df['path'] + "/"
s += df['filename'].str[:-4] + "_000_inferred.tif"
df['infermask'] = s

for tmp in s:
    if not os.path.exists(tmp):
        print(tmp)

In [10]:
df['rdna_green_ratio'] = (df.rdna_mean/df.green_mean)
df['rdna_vol_ratio'] = df.rdna_vol/df.vol
df['red_vol_ratio'] = df.red_voxels/df.vol
df['blue_green_ratio'] = df['blue_mean']/df['green_mean']
df['blue_green_vol_ratio'] = df['blue_vol']/df['vol']
df['blue_red_ratio'] = df['blue_mean']/df['green_mean']

In [12]:
def get_xy_label(x):
    _image = tifffile.imread(x['infermask'].values[0])
    image = clear_border(_image, buffer_size=1)
    mask_name = x['infermask'].values[0]
    mask_name = mask_name.replace(".tif", "_no_border.tif")
    tifffile.imwrite(mask_name, image)
    df_list = list()
    index_list = list()
    label_list = list()
    for row in x.itertuples():
        index_list.append(row.Index),
        label_list.append(image[row.ycm, row.xcm])
    
    return pd.DataFrame({'label_val':label_list, 'label_index':index_list})

In [13]:
label_df = df.groupby('file')[['infermask','xcm', 'ycm']].apply(get_xy_label).reset_index()

In [ ]:
merge_df = df.merge(label_df, left_index=True, right_on='label_index', how='left').reset_index()

In [24]:
filtdf = merge_df[merge_df['cell_length'] > 0].copy()
filtdf = filtdf[filtdf['green_mean'] > 5].copy()
filtdf = filtdf[filtdf['blue_mean'] > 5].copy()
print(len(filtdf), len(merge_df))

184 580


In [25]:
filtdf['num_nuclei'] = filtdf.groupby(['file_x', 'label_val'])['vol'].transform(len)

In [27]:
filtdf.to_csv(f'output_data.csv')

In [ ]:
from plotly import express as px
from plotly import graph_objects as go

fig = px.histogram(filtdf, x="red_voxels", width=600, height=600,
                   color='mutant',
                    barmode='overlay', histnorm='percent'
                   )
fig